# Проектная работа "Исследовательский анализ данных"

### Описание проекта
В нашем распоряжении данные сервиса **Яндекс.Недвижимость** — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. 
Цель - определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

---
### Шаг 1. Знакомство с данными

Путь к файлу: /datasets/real_estate_data.csv

https://code.s3.yandex.net/datasets/real_estate_data.csv


In [2]:
from textwrap import dedent
from IPython import get_ipython

In [3]:
import pandas as pd

# Подключаем источник данных
#df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
df = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv', sep='\t')

# Получаем информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

In [4]:
# Размер таблицы
print(f"В таблице {df.shape[1]} столбцов и {df.shape[0]} строк.")

В таблице 22 столбцов и 23699 строк.


In [5]:
# Выведем пять строк таблицы
df.head()

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


**Описание столбцов**

В исходных данных даны следующие параметры столбцов:

| Имя столбца | Описание данных в столбце | Размерность | Тип данных |
| --- | --- | --- | --- |
| `total_images` | Количество фотографий в объявлении | шт. | integer |
| `last_price` | Цена на момент снятия с публикации | ден.ед. | float |
| `total_area` | Площадь квартиры | м² | float |
| `first_day_exposition` | Дата публикации объявления | дата | datetime |
| `rooms` | Количество комнат | шт. | integer |
| `ceiling_height` | Высота потолков | м | float |
| `floors_total` | Количество этажей в здании | шт. | integer |
| `living_area` | Жилая площадь | м² | float |
| `floor` | Этаж | - | integer |
| `is_apartment` | Тип жилого помещения апартаменты | - | boolean |
| `studio` | Тип жилого помещения студия | - | boolean |
| `open_plan` | Свободная планировка жилого помещения | - | boolean |
| `kitchen_area` | Площадь кухни | м² | float |
| `balcony` | Количество балконов | шт. | integer |
| `locality_name` | Название населённого пункта | - | string |
| `airports_nearest` | Расстояние до ближайшего аэропорта | м | integer |
| `cityCenters_nearest` | Расстояние до центра города | м | integer |
| `parks_around3000` | Число парков в радиусе 3 км | шт. | integer |
| `parks_nearest` | Расстояние до ближайшего парка | м | integer |
| `ponds_around3000` | Число водоёмов в радиусе 3 км | шт. | integer |
| `ponds_nearest` | Расстояние до ближайшего водоёма | м | integer |
| `days_exposition` | Количество дней публикации объявления | шт. | integer |

**Вывод**

1. Наименования столбцов: некоторые столбцы содержат смешенный стиль названия, например, `cityCenters_nearest`. А некторые названия можно уточнить или доплнить для удобства последующей работы.


2. Тип данных в столбцах: часто встречаются вещественные типы данных, тогда как в столбце должен быть целочисленный тип. Это встречается в следующих столбцах:

<ul style='list-style-type: none; margin-left: 50px;'>
<li><code>floors_total</code>,</li>
<li><code>balcony</code>,</li>
<li><code>airports_nearest</code>,</li>
<li><code>cityCenters_nearest</code>,</li>
<li><code>parks_around3000</code>,</li>
<li><code>parks_nearest</code>,</li>
<li><code>ponds_around3000</code>,</li>
<li><code>ponds_nearest</code>,</li>
<li><code>days_exposition</code>,</li>
    </ul>

3. Неполнота данных: в таких столбцах пропущены данные:

<ul style='list-style-type: none; margin-left: 50px;'>
<li><code>ceiling_height</code>,</li>
<li><code>floors_total</code>,</li>
<li><code>living_area</code>,</li>
<li><code>is_apartment</code>,</li>
<li><code>kitchen_area</code>,</li>
<li><code>balcony</code>,</li>
<li><code>locality_name</code>,</li>
<li><code>airports_neares</code>,</li>
<li><code>cityCenters_nearest</code>,</li>
<li><code>parks_around3000</code>,</li>
<li><code>parks_nearest</code>,</li>
<li><code>ponds_around3000</code>,</li>
<li><code>ponds_nearest</code>,</li>
<li><code>days_exposition</code>,</li>
    </ul>

---
### Шаг 2. Предобработка данных


2.0 Переименование столбцов

Перед началом работы приведём названия некоторых столбцов к следующему виду:

<ul style='list-style-type: none; margin-left: 50px;'>
<li><code>studio</code> → <code>is_studio</code>,</li>
<li><code>open_plan</code> → <code>is_open_plan</code>,</li>
<li><code>cityCenters_nearest</code> → <code>center_distance</code>,</li>
<li><code>parks_around3000</code> → <code>parks_around</code>,</li>
<li><code>ponds_around3000</code> → <code>ponds_around</code>.</li>
    </ul>

In [8]:
df = df.rename(
    columns={
        'studio': 'is_studio',
        'open_plan': 'is_open_plan',
        'cityCenters_nearest': 'center_distance',
        'parks_around3000': 'parks_around',
        'ponds_around3000': 'ponds_around'
    })

1.   Определите и изучите пропущенные значения:
    

*   Для некоторых пропущенных значений можно предположить логичную замену.
Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски правильно заменить на 0. Для других типов данных нет подходящего значения на замену. В этом случае правильно оставить эти значения пустыми. Отсутствие значения — тоже важный сигнал, который не нужно прятать.

*   Заполните пропуски, где это уместно. Опишите, почему вы решили заполнить пропуски именно в этих столбцах и как выбрали значения.

*   Укажите причины, которые могли привести к пропускам в данных.

In [9]:
df.columns

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'is_studio', 'is_open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'center_distance', 'parks_around',
       'parks_nearest', 'ponds_around', 'ponds_nearest', 'days_exposition'],
      dtype='object')

2.   Приведите данные к нужным типам: Поясните, в каких столбцах нужно изменить тип данных и почему.

- Первично предобработаю столбцы. <= 20 * 22 минут = 7 часов 20 минут
    * Изучу на предмет пропуков, неуместных значений и неподходящих типов. 3 минут на каждый столбец
    * Заполню вывод 10 минут на каждый столбец:
        - Обнаруженные аномалии: ...
        - Причины, по которым они могли возникнуть: ...
        - Как исправлять: ...
        - Финальный тип и почему: ...
    * Удалю строки, если нужно. 7 минут
- Заполню пропуски после удаления выбросов. 10 минут

In [23]:
def create_cell(shell, text):
    payload = dict(
        source='set_next_input',
        text=text,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)

def generate_column_preprocess_description(df):
    shell = get_ipython()
    for i, column in reversed(list(enumerate(df.columns))):
        create_cell(shell, dedent(
            '''
            * Изменение
            '''
        ))
        create_cell(shell, dedent(
            '''
            * Вывод
            \t- Обнаруженные аномалии: ...
            \t- Причины, по которым они могли возникнуть: ...
            \t- Как исправлять: ...
            \t- Финальный тип и почему: ...
            '''
        ))
        create_cell(shell, dedent(
            f'''
            investigate_column(df["{column}"])
            '''
        ))
        create_cell(shell, dedent(
            '''
            * Изучим значения
            '''
        ))
        create_cell(shell, dedent(
            f'''
            #### {i+1}. Обработка "{column}"
            '''
        ))

In [24]:
def investigate_column(column):
    print('Null or Na count is', column.isna().sum())
    print(column.describe())

In [29]:
generate_column_preprocess_description(df)

None


---

### Шаг 3. Посчитайте и добавьте в таблицу
  
3.1   цену квадратного метра;

3.2   день недели, месяц и год публикации объявления;

3.3   этаж квартиры; варианты — первый, последний, другой;

3.4   соотношение жилой и общей площади, а также отношение площади кухни к общей.

**Новые признаки. 40 минут:**


3.1 Поделю `last_price` на `total_area` и посчитаю квадртный метр (`price_per_area`) 5 минут

3.2 Извлеку из поля `first_day_exposition` день недели (`first_exposition_day_of_week`), месяц (`first_exposition_month`) и год публикации объявления (`first_exposition_year`) 10 минут

3.3 Напишу функцию `get_floor_type`, которая по floor возвращает тип: первый (first), последний (last), другой (other). И применю её к столбцу `floor`, результат запишу в столбец `floor_type`: 10 минут

3.4 Посчитаю соотношение жилой и общей площади. Для этого поделю `living_area` на `total_area`. Результат сохраню в `living_total_ratio`. 5 минут

Посчитаю отношение площади кухни к общей. Для этого поделю `kitchen_area` на `total_area`. Результат сохраню в `living_total_ratio`. 5 минут

---

### Шаг 4. Проведите исследовательский анализ данных.

#### 4.1 Изучите следующие параметры: площадь, цена, число комнат, высота потолков. Постройте гистограммы для каждого параметра.



- Построю гистограмму для `total_area`, `last_price`, `rooms` и `ceiling_height` с помощью матрицы диаграмм рассеяния. А так же построю ящик с усами. 15 минут
- Опишу в промежуточном выводе, что интересного заметил 15 минут


#### 4.2 Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?



- Для поля `days_exposition` построю гистограмму и ящик с усами. А также посчитаю выборочную медиану и среднее. 10 минут
- Отвечу в выводе на вопросы 15 минут



#### 4.3 Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили.


- Опишу все подозрения, которые возникли за это время 20 минут
- Удалю строки с аномальными значениями 10 минут

#### 4.4 Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от площади, числа комнат, удалённости от центра. Изучите зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом. Также изучите зависимость от даты размещения: дня недели, месяца и года.


- Построю диаграмму рассеяния для пар (`last_price`, `total_area`), (`last_price`, `rooms`), (`last_price`, `cityCenters_nearest`), (`last_price`, `first_exposition_day_of_week`), (`last_price`, `first_exposition_month`), (`last_price`, `first_exposition_year`). А также пострю ящик с усами для `last_price` по всем возможным `floor_type`. 30 минут
- Опишу в выводе сначала то, где есть зависимость от цены. А далее отвечу на вопрос, какой фактор влияет сильнее. 15 минут

#### 4.5 Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите среди них населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце `locality_name`.


- Через pivot_table выберу 10 населенные пунктов с наибольшим числом объявлений. Далее через filter и pivot_table рассчитаю `avg_price_per_area` по полю `price_per_area`. 15 минут
- Опишу в выводе какие из населенных пунктов имеют самую высокую и самую низкую стоимость жилья. 10 минут

#### 4.6 Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге `locality_name`. Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется, — это и будет центральная зона.


- Выделю данные для Питера в переменную `df_spb`. 5 минут
- С помощью pivot_table посчитаю среднюю `price_per_area` и среднюю `last_price` с группировкой по `center_distance_km`. Назову их `avg_price_per_area` и `avg_last_price`. 10 минут
- Построю графики зависимостей `avg_price_per_area` и `avg_last_price` от center_distance_km 10 минут
- Опишу в выводе где проходит граница. Объясню это резким увеличением в цене. 10 минут

#### 4.7 Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всему городу?


- Из переменной `df_spb` выделю только центральные объявления в переменную `df_spb_center`. 5 минут
- Построю гистограмму по `df_spb_center` для `total_area`, `last_price`, `rooms` и `ceiling_height` с помощью матрицы диаграмм рассеяния. А так же построю ящик с усами. 15 минут
- Построю по `df_spb_center` диаграмму рассеяния для пар (`last_price`, `total_area`), (`last_price`, `rooms`), (`last_price`, `cityCenters_nearest`), (`last_price`, `first_exposition_day_of_week`), (`last_price`, `first_exposition_month`), (`last_price`, `first_exposition_year`). А также пострю ящик с усами для `last_price` по всем возможным `floor_type`. 30 минут
- В выводе опишу сначала какие факторы влияют на цену. 10 минут
- А проведу сравнением с значениями по всему датасету 30 минут

---

### Шаг 5. Общий вывод

- Напишу общий вывод. Расскажу сперва про аномалии. А далее про зависимости, которые были замечены. И что с ними произошло, когда мы рассмотрели срез центральных объявлений. 1 час

---

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [ ]  файлы изучены (выведены первые строки, метод info())
- [ ]  определены пропущенные значения
- [ ]  заполнены пропущенные значения
- [ ]  есть пояснение, какие пропущенные значения обнаружены
- [ ]  изменены типы данных
- [ ]  есть пояснение, в каких столбцах изменены типы и почему
- [ ]  посчитано и добавлено в таблицу: цена квадратного метра
- [ ]  посчитано и добавлено в таблицу: день недели, месяц и год публикации объявления
- [ ]  посчитано и добавлено в таблицу: этаж квартиры; варианты — первый, последний, другой
- [ ]  посчитано и добавлено в таблицу: соотношение жилой и общей площади, а также отношение площади кухни к общей
- [ ]  изучены следующие параметры: площадь, цена, число комнат, высота потолков
- [ ]  построены гистограммы для каждого параметра
- [ ]  выполнено задание: "Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?"
- [ ]  выполнено задание: "Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили."
- [ ]  выполнено задание: "Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от квадратного метра, числа комнат, этажа (первого или последнего), удалённости от центра. Также изучите зависимость от даты размещения: дня недели, месяца и года. "Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце '*locality_name'*. "
- [ ]  выполнено задание: "Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (*'locality_name'*). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется — это и будет центральная зона. "
- [ ]  выполнено задание: "Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всему городу?"
- [ ]  в каждом этапе есть выводы
- [ ]  есть общий вывод